In [4]:
using CairoMakie
using Oceananigans
using Interpolations
using Statistics

using Meshing
using MeshIO
using GeometryBasics

include("../src-fig/plotting.jl")

# Mesh, negative inside
function create_isosurface_mesh(filename,
        field,
        X, Y, Z,
    )
    
    output = ones(eltype(field), size(field) .+ 2)
    output[2:end-1, 2:end-1, 2:end-1] .= field
    
    alg = MarchingTetrahedra(iso=0.0)
    points, faces = isosurface(output, alg, X, Y, Z)
    msh = GeometryBasics.Mesh(GeometryBasics.Point.(points), SimplexFace.(faces))
    save(filename, msh)
    return nothing
end

function create_isosurface_mesh(filename,
        field;
        x_resolution=1,
        y_resolution=1,
        z_resolution=1
    )
    X = x_resolution .* (0:(size(field, 1) + 1))
    Y = y_resolution .* (0:(size(field, 2) + 1))
    Z = z_resolution .* (0:(size(field, 3) + 1))
    
    create_isosurface_mesh(filename, field, X, Y, Z)
end

# Heightmap
function create_heightmap_mesh(filename,
        field;
        x_resolution=1,
        y_resolution=1,
        z_resolution=1,
        z_range=(10, 20)
    )
    # normalise field to z_range
    field = (field .- minimum(field)) ./ (maximum(field) - minimum(field))
    field = z_range[1] .+ (z_range[2] - (z_range[1] + 3)) .* field
    zs = range(z_range..., 100)
    data = [zs[k] - field[i, j] for i in 1:size(field, 1), j in 1:size(field, 2), k in 1:length(zs)]
    create_isosurface_mesh(filename, data; x_resolution, y_resolution, z_resolution)
end

create_heightmap_mesh (generic function with 1 method)

In [51]:
foldername = "../../scratch/filament-instability-redux/Ri/Ri00"
iterations, times = iterations_times(foldername)
xsᶜ, xsᶠ, ysᶜ, ysᶠ, zsᶜ, zsᶠ = grid_nodes(foldername)
field = timeseries_of(joinpath(foldername, "DFM.jld2"), "b_dfm", iterations) .+ 240;
field = filt(field, 1, 1, 0)
create_isosurface_mesh("NIEchoes.obj", field;
    x_resolution=0.5,
    y_resolution=0.25,
    z_resolution=0.5,
)

In [5]:
foldername = "../../scratch/turbulence-at-fronts/StrainQ1"
iterations, times = iterations_times(foldername)
sp = simulation_parameters(foldername)
xsᶜ, xsᶠ, ysᶜ, ysᶠ, zsᶜ, zsᶠ = grid_nodes(foldername)
inds = centre_indices(foldername)
frames = [1, 31, 157, 278, 400, 500, 600, 700, 800];

In [9]:
u_series = timeseries_of(u->u[inds, 20], joinpath(foldername, "DFM.jld2"), "u_dfm", iterations)
create_heightmap_mesh("u_20.obj", filt(u_series, 4)[1:4:end, 1:4:end]; z_range=(zsᶜ[17], zsᶜ[23]))

u_series = timeseries_of(u->u[inds, 28], joinpath(foldername, "DFM.jld2"), "u_dfm", iterations)
create_heightmap_mesh("u_28.obj", filt(u_series, 4)[1:4:end, 1:4:end]; z_range=(zsᶜ[25], zsᶜ[31]))

In [3]:
map(401:50:601) do frame
    println(frame)
    get_field(joinpath(foldername, "output.jld2"), "v", iterations[frame]) do field
        field = filt(field[inds, :, :] .- mean(field[inds, :, end]), 1, 1, 0)
        create_isosurface_mesh(joinpath("snapshots", "$frame.obj"), -field, 0.1, 0.1)
    end
end

401
451
501
551
601


5-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing